# Chapter 03. Interactive Model Training

In this chapter we will demonstrate how to train models using built-in algorithms as well as custom code.

## 3.1 Built-in Algorithms
---

### 3.1.1 Setup
---

In [2]:
%cd /root/sagemaker-course/notebooks/

/root/sagemaker-course/notebooks


In [15]:
import boto3
import pandas as pd

pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 10)         # Keep the output on one page

BUCKET = 'sagemaker-course-20200517'
PREFIX = 'churn'
LOCAL_DATA_DIRECTORY = f'../data/{PREFIX}'

print(f"Artifacts will be written to s3://{BUCKET}/{PREFIX}")

Artifacts will be written to s3://sagemaker-course-20200517/churn


In [4]:
import sagemaker

In [20]:
sagemaker_session = sagemaker.Session()
boto_session = sagemaker_session.boto_session

In [13]:
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

arn:aws:iam::209970524256:role/service-role/AmazonSageMaker-ExecutionRole-20200414T065516


### 3.1.2 Data
---

To see how the dataset was preprocessed, see this notebook: [XGBoost customer churn notebook that starts with the original dataset](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_applying_machine_learning/xgboost_customer_churn/xgboost_customer_churn.ipynb). 

In [16]:
local_data_path = f'{LOCAL_DATA_DIRECTORY}/training-dataset-with-header.csv'
data = pd.read_csv(local_data_path)

data

,Churn,Account Length,VMail Message,Day Mins,Day Calls,Eve Mins,Eve Calls,Night Mins,Night Calls,Intl Mins,Intl Calls,CustServ Calls,State_AK,State_AL,State_AR,State_AZ,State_CA,State_CO,State_CT,State_DC,State_DE,State_FL,State_GA,State_HI,State_IA,State_ID,State_IL,State_IN,State_KS,State_KY,State_LA,State_MA,State_MD,State_ME,State_MI,State_MN,State_MO,State_MS,State_MT,State_NC,State_ND,State_NE,State_NH,State_NJ,State_NM,State_NV,State_NY,State_OH,State_OK,State_OR,State_PA,State_RI,State_SC,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY,Area Code_408,Area Code_415,Area Code_510,Int'l Plan_no,Int'l Plan_yes,VMail Plan_no,VMail Plan_yes
0,0,106,0,274.4,120,198.6,82,160.8,62,6.0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
1,0,28,0,187.8,94,248.6,86,208.8,124,10.6,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0
2,1,148,0,279.3,104,201.6,87,280.8,99,7.9,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0
3,0,132,0,191.9,107,206.9,127,272.0,88,12.6,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
4,0,92,29,155.4,110,188.5,104,254.9,118,8.0,4,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2328,0,106,0,194.8,133,213.4,73,190.8,92,11.5,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0
2329,1,125,0,143.2,80,88.1,94,233.2,135,8.8,7,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
2330,0,129,0,143.7,114,297.8,98,212.6,86,11.4,8,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0
2331,0,159,0,198.8,107,195.5,91,213.3,120,16.5,7,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0


In [19]:
s3_input_train = sagemaker_session.upload_data(f'{LOCAL_DATA_DIRECTORY}/train.csv',
                                               bucket=BUCKET,
                                               key_prefix=PREFIX)

s3_input_validation = sagemaker_session.upload_data(f'{LOCAL_DATA_DIRECTORY}/validation.csv',
                                                    bucket=BUCKET,
                                                    key_prefix=PREFIX)

s3_input_train = sagemaker.s3_input(s3_data=s3_input_train, content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data=s3_input_validation, content_type='csv')

### 3.1.3 Training
---

In [21]:
from sagemaker.amazon.amazon_estimator import get_image_uri

xgboost_image_name = get_image_uri(boto_session.region_name, 'xgboost', repo_version='0.90-2')
xgboost_image_name

'257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3'

In [25]:
from sagemaker import estimator

In [26]:
xgb_model = estimator.Estimator(image_name=xgboost_image_name,
                                role=role,
                                train_instance_count=1,
                                train_instance_type='ml.m4.xlarge',
                                output_path=f"s3://{BUCKET}/{PREFIX}",
                                base_job_name="builtin-xgboost",
                                sagemaker_session=sagemaker_session)

In [27]:
xgb_model.set_hyperparameters(max_depth=5,
                              subsample=0.8,
                              num_round=600,
                              eta=0.2,
                              gamma=4,
                              min_child_weight=6,
                              silent=0,
                              objective='binary:logistic')

In [ ]:
xgb_model.fit({'train': s3_input_train,
               'validation': s3_input_validation})

## 3.2 Custom Code with Pre-built Docker Images
---

### 3.2.1 Create a custom Scikit-learn script to train a model
---

See the [sagemaker-training-toolkit](https://github.com/aws/sagemaker-training-toolkit/blob/master/ENVIRONMENT_VARIABLES.md) repository for a comphrehensive list of environment variables used by SageMaker Docker images.

In [59]:
!pygmentize '../scripts/sklearn/sklearn_rf.py'

from __future__ import print_function

import argparse
import os
import pandas as pd

from sklearn import ensemble
from sklearn.externals import joblib


if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    # Hyperparameters are described here. In this simple example we are just including one hyperparameter.
    parser.add_argument('--n_estimators', type=int, default=100)

    # Sagemaker specific arguments. Defaults are set in the environment variables.
    parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])

    args = parser.parse_args()

    # Take the set of files and read them all into a single pandas dataframe
    input_files = [ os.path.join(args.train, file) for file in os.listdir(args.train) ]
    if len(input_files) == 0:
        raise ValueError(('T

### 3.2.2 Train a SageMaker Scikit Estimator
---

In [34]:
from sagemaker.sklearn.estimator import SKLearn

sklearn_estimator = SKLearn(
    entry_point='../scripts/sklearn/sklearn_rf.py',
    code_location=f's3://{BUCKET}/{PREFIX}',
    hyperparameters={'n_estimators': 50},
    role=role,
    train_instance_type='ml.c4.xlarge',
    output_path=f's3://{BUCKET}/{PREFIX}',
    base_job_name="custom-code-sklearn",
    sagemaker_session=sagemaker_session)

In [36]:
sklearn_estimator.fit({'train': s3_input_train})

2020-05-17 14:49:12 Starting - Starting the training job...
2020-05-17 14:49:41 Starting - Launching requested ML instances.........
2020-05-17 14:50:43 Starting - Preparing the instances for training...
2020-05-17 14:51:33 Downloading - Downloading input data...
2020-05-17 14:52:03 Training - Downloading the training image..2020-05-17 14:52:16,967 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-05-17 14:52:16,969 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-05-17 14:52:16,980 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-05-17 14:52:17,240 sagemaker-containers INFO     Module sklearn_rf does not provide a setup.py. 
Generating setup.py
2020-05-17 14:52:17,240 sagemaker-containers INFO     Generating setup.cfg
2020-05-17 14:52:17,240 sagemaker-containers INFO     Generating MANIFEST.in
2020-05-17 14:52:17,241 sagemaker-containers INFO     Installing module with the follow

In [42]:
# Head to the URL in the following cell to view the details of this training job.

f"https://{boto_session.region_name}.console.aws.amazon.com/sagemaker/home?region={boto_session.region_name}#/jobs/{sklearn_estimator.jobs[0].job_name}"

'https://us-east-2.console.aws.amazon.com/sagemaker/home?region=us-east-2#/jobs/custom-code-sklearn-2020-05-17-14-49-12-285'

### 3.3 Installing custom Python requirements
---

In [47]:
sklearn_estimator = SKLearn(
    entry_point='../scripts/sklearn/sklearn_rf.py',
    code_location=f's3://{BUCKET}/{PREFIX}',
    hyperparameters={'n_estimators': 50},
    role=role,
    train_instance_type='ml.c4.xlarge',
    output_path=f's3://{BUCKET}/{PREFIX}',
    base_job_name="custom-code-sklearn",
    sagemaker_session=sagemaker_session)

print(sklearn_estimator.image_name)

257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3


The [sagemaker-scikit-learn-container](https://github.com/aws/sagemaker-scikit-learn-container/) repo contains the source code, including the Dockerfile, for creating this image.

In [48]:
!cat ../scripts/requirements.txt

eli5

In [57]:
sklearn_estimator = SKLearn(
    entry_point='sklearn_rf.py',
    source_dir='../scripts/sklearn',
    code_location=f's3://{BUCKET}/{PREFIX}',
    hyperparameters={'n_estimators': 50},
    role=role,
    train_instance_type='ml.c4.xlarge',
    output_path=f's3://{BUCKET}/{PREFIX}',
    base_job_name="install-libs-sklearn",
    sagemaker_session=sagemaker_session)

In [58]:
sklearn_estimator.fit({'train': s3_input_train})

2020-05-17 17:26:49 Starting - Starting the training job...
2020-05-17 17:26:51 Starting - Launching requested ML instances......
2020-05-17 17:27:54 Starting - Preparing the instances for training...
2020-05-17 17:28:32 Downloading - Downloading input data...
2020-05-17 17:29:15 Training - Training image download completed. Training in progress..2020-05-17 17:29:16,523 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-05-17 17:29:16,525 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-05-17 17:29:16,536 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-05-17 17:29:16,799 sagemaker-containers INFO     Module sklearn_rf does not provide a setup.py. 
Generating setup.py
2020-05-17 17:29:16,799 sagemaker-containers INFO     Generating setup.cfg
2020-05-17 17:29:16,800 sagemaker-containers INFO     Generating MANIFEST.in
2020-05-17 17:29:16,800 sagemaker-containers INFO     Installing 